In [ ]:
#imports
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis
import wfdb
from wfdb import processing
import getopt
import sys
import os
import glob
from ripser import ripser, plot_dgms 
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score


In [ ]:
#functions, maybe save this as a different python file and just call

cwd = os.getcwd() 
#input is ecg ID, outputs a dictionary with two keys: Data and RRstats, both of which contain arrays
def ExtractDataAndRRFeatures(X):
    Data, fields_Total = wfdb.rdsamp('test/'+X)
    if (skew(Data) < 0 ):
        Data = -Data
    xqrs_Total = processing.XQRS(sig =Data[:,0], fs = fields_Total["fs"])
    xqrs_Total.detect()
    Time_Total    = xqrs_Total.qrs_inds
    Time_1 = Time_Total[0:len(Time_Total)-1]
    Time_2 = Time_Total[1:]
    RR_interval  = Time_2 - Time_1
    
    if( len(Time_1) > 2 ):
        Mean_RR_interval = np.mean(RR_interval)
        SD_RR_interval   = np.std(RR_interval)
        Kurt_RR_interval = kurtosis(RR_interval)
        Skew_RR_interval = skew(RR_interval)
    else:
        mean_RR_interval = SD_RR_interval = Kurt_RR_interval = skew_RR_interval = 1000
    return dict([("Data", Data), ('RRstats',np.array([Mean_RR_interval,SD_RR_interval, Kurt_RR_interval,Skew_RR_interval]))])
        
    
def Clean(X):
    #insert Chris's or Datta's cleaning algo
    #Cleaned = np.array([])

    
    #this next lines is a proxy for the cleanup to check if the code is running
    Cleaned = []
    with open(X) as f:
        for line in f:
            data = line.split()
            Cleaned.append(float(data[0]))
    return Cleaned

def NoisyCheck(CleanData,DictionaryData):
    if (len(CleanData)/len(DictionaryData["Data"])<=0.2):
        if (DictionaryData["RRstats"][1]>SDtresh):
            return 1
        else:
            return 0
    else:
        return 0
    
def extract(CleanData):
    ALL = np.round(np.arange(0,len(CleanData),step))
    PTcloud = np.array(np.zeros((len(ALL)-DIM+1)*DIM))
    PTcloud.shape = (len(ALL)-DIM+1,DIM)
    for i in range(len(ALL)-DIM):
         ALL2 = ALL[range(i,(DIM+i))]
         ALL2 = ALL2.astype(int)
         PTcloud[i] = np.array([CleanData[k] for k in ALL2])
    while (sum(abs(PTcloud[len(PTcloud)-1]))==0):
         PTcloud = PTcloud[0:len(PTcloud)-1]
    diagrams = ripser(PTcloud, maxdim=maxdimension)['dgms']
    N1 = N2 = MeanBC1 = MeanDC1 = SUMpersC1 = SDBC1 = SDDC1 = SDPersC1 = SKPersC1 = SUMpersC1 = SKBC1 = KurtBC1 = 0
    SKDC1 = KurtDC1 = MeanpersC1 = KurtpersC1 = MeanBC2 = MeanDC2 = SUMpersC2 = SDBC2 = SDDC2 = SDPersC2 = SKPersC2 = 0
    SUMpersC2 = SKBC2 = KurtBC2 = SKDC2 = KurtDC2 = MeanpersC2 = KurtpersC2 = 0
    N0 = len(diagrams[0])  
    C0d = np.array(diagrams[0][:len(diagrams[0])-1,1])
    MeanDC0 = np.mean(C0d)
    SUMpersC0 = sum(C0d)
    SDDC0 = np.std(C0d)
    SKDC0 = skew(C0d)
    KurtDC0 = kurtosis(C0d)
    d0 = diagrams[0][:len(diagrams[0])-1,1]
    hd0 = np.histogram(d0,bins = 'auto')
    d0 = d0[:len(d0)-1-len(np.where(d0>=hd0[1][len(hd0[1])-2])[0])]

    MeanDC0noPers = np.mean(d0)
    SDDC0noPers = np.std(d0)
    SKDC0noPers = skew(d0)
    KurtDC0noPers = kurtosis(d0)
    MeanBC1noPers = SDBC1noPers = SKBC1noPers = KurtBC1noPers = MeanDC1noPers = SDDC1noPers = SKDC1noPers = KurtDC1noPers = 0
    SUMPersC1noPers = MeanPersC1noPers = SDPersC1noPers = SKPersC1noPers = KurtPersC1noPers = 0
    MeanBC2noPers = SDBC2noPers = SKBC2noPers = KurtBC2noPers = MeanDC2noPers = SDDC2noPers = SKDC2noPers = KurtDC2noPers = 0
    SUMPersC2noPers = MeanPersC2noPers = SDPersC2noPers = SKPersC2noPers = KurtPersC2noPers = 0

    if(len(diagrams[1])>0):
        N1 = len(diagrams[1])
        d0=diagrams[1][:len(diagrams[1])-1,0]
        d1=diagrams[1][:len(diagrams[1])-1,1]
        d = d1-d0
        if(N1>1):
            C1b = np.array(diagrams[1][:,0])
            C1d = np.array(diagrams[1][:,1])
            MeanBC1 = np.mean(C1b)
            MeanDC1 = np.mean(C1d)
            SUMpersC1 = sum(C1d-C1b)
            SDBC1 = np.std(C1b)
            SDDC1 = np.std(C1d)
            SDPersC1 = np.std(C1d-C1b)
            SKPersC1 = skew(C1d-C1b)
            SKBC1 = skew(C1b)
            KurtBC1 = kurtosis(C1b)
            SKDC1 = skew(C1d)
            KurtDC1 = kurtosis(C1d)
            MeanpersC1 = np.mean(C1d-C1b)
            KurtpersC1 = kurtosis(C1d-C1b)
            hd = np.histogram(d,bins = 'auto')
            d = d[:len(d)-1-len(np.where(d>=hd[1][len(hd[1])-2])[0])]
            MeanBC1noPers = np.mean(d0)
            SDBC1noPers = np.std(d0)
            SKBC1noPers = skew(d0)
            KurtBC1noPers = kurtosis(d0)
            MeanDC1noPers = np.mean(d1)
            SDDC1noPers = np.std(d1)
            SKDC1noPers = skew(d1)
            KurtDC1noPers = kurtosis(d1)
            SUMPersC1noPers = sum(d)
            MeanPersC1noPers = np.mean(d)
            SDPersC1noPers = np.std(d)
            SKPersC1noPers = skew(d)
            KurtPersC1noPers = kurtosis(d)
        else:
            MeanBC1 = diagrams[1][:,0]
            MeanDC1 = diagrams[1][:,1]
            SUMpersC1 = diagrams[1][:,1]-diagrams[1][:,0]
            SDBC1 = SDDC1 = SDPersC1 = SKPersC1 = SKBC1 = KurtBC1 = SKDC1 = KurtDC1 = KurtpersC1 = 100
            MeanpersC1 = diagrams[1][:,1]-diagrams[1][:,0]

            MeanBC1noPers = SDBC1noPers = SKBC1noPers = KurtBC1noPers = MeanDC1noPers = SDDC1noPers = SKDC1noPers = 100
            KurtDC1noPers = SUMPersC1noPers = MeanPersC1noPers = SDPersC1noPers = SKPersC1noPers = KurtPersC1noPers = 100
            
    if(len(diagrams)==3):
        N2 = len(diagrams[2])
        d0=diagrams[2][:len(diagrams[2])-1,0]
        d1=diagrams[2][:len(diagrams[2])-1,1]
        d = d1-d0
        if N2>1:

            C2b = np.array(diagrams[2][:,0])
            C2d = np.array(diagrams[2][:,1])
            MeanBC2 = np.mean(C2b)
            MeanDC2 = np.mean(C2d)
            SUMpersC2 = sum(C2d-C2b)
            SDBC2 = np.std(C2b)
            SDDC2 = np.std(C2d)
            SDPersC2 = np.std(C2d-C2b)
            SKPersC2 = skew(C2d-C2b)
            SKBC2 = skew(C2b)
            KurtBC2 = kurtosis(C2b)
            SKDC2 = skew(C2d)
            KurtDC2 = kurtosis(C2d)
            MeanpersC2 = np.mean(C2d-C2b)
            KurtpersC2 = kurtosis(C2d-C2b)

            hd = np.histogram(d,bins = 'auto')
            d = d[:len(d)-1-len(np.where(d>=hd[1][len(hd[1])-2])[0])]
            MeanBC2noPers = np.mean(d0)
            SDBC2noPers = np.std(d0)
            SKBC2noPers = skew(d0)
            KurtBC2noPers = kurtosis(d0)
            MeanDC2noPers = np.mean(d1)
            SDDC2noPers = np.std(d1)
            SKDC2noPers = skew(d1)
            KurtDC2noPers = kurtosis(d1)
            SUMPersC2noPers = sum(d)
            MeanPersC2noPers = np.mean(d)
            SDPersC2noPers = np.std(d)
            SKPersC2noPers = skew(d)
            KurtPersC2noPers = kurtosis(d)
        elif (N2==1):
            #print("Merun C1")
            MeanBC2 = diagrams[2][:,0]
            MeanDC2 = diagrams[2][:,1]
            SUMpersC2 = diagrams[2][:,1]-diagrams[2][:,0]
            SDBC2 = SDDC2 = SDPersC2 = SKPersC2 = SKBC2 = KurtBC2 = SKDC2 = KurtDC2 = KurtpersC2 = 1000
            MeanpersC2 = diagrams[2][:,1]-diagrams[2][:,0]
            
            MeanBC2noPers = SDBC2noPers = SKBC2noPers = KurtBC2noPers = MeanDC2noPers = SDDC2noPers = SKDC2noPers = 1000
            KurtDC2noPers = SUMPersC2noPers = MeanPersC2noPers = SDPersC2noPers = SKPersC2noPers = KurtPersC2noPers = 1000
    #trim these features to have the same as the random forest
    #ECGFeat = np.array([N0, MeanDC0, SDDC0, SUMpersC0, SKDC0, KurtDC0, MeanBC1, MeanDC1, SDBC1, SDDC1, SUMpersC1, SKPersC1,N1, SDPersC1, SKBC1, KurtBC1,SKDC1,KurtDC1,MeanpersC1,KurtpersC1,  N2, SDPersC2,MeanBC2, MeanDC2, SDBC2, SDDC2,SKPersC2, SUMpersC2,SKBC2, KurtBC2,SKDC2,KurtDC2,MeanpersC2,KurtpersC2,MeanDC0noPers, SDDC0noPers,SKDC0noPers,KurtDC0noPers,MeanBC1noPers,SDBC1noPers,SKBC1noPers,KurtBC1noPers,MeanDC1noPers,SDDC1noPers,SKDC1noPers,KurtDC1noPers,SUMPersC1noPers,MeanPersC1noPers,SDPersC1noPers,SKPersC1noPers, KurtPersC1noPers,MeanBC2noPers,SDBC2noPers,SKBC2noPers,KurtBC2noPers,MeanDC2noPers,SDDC2noPers,SKDC2noPers,KurtDC2noPers,SUMPersC2noPers,MeanPersC2noPers,SDPersC2noPers,SKPersC2noPers, KurtPersC2noPers])
    ECGFeat = np.array([N0, MeanDC0, SDDC0, SUMpersC0, SKDC0, KurtDC0, MeanBC1, MeanDC1, SDBC1, SDDC1, SUMpersC1, SKPersC1,N1, SDPersC1, SKBC1, KurtBC1,SKDC1,KurtDC1,MeanpersC1,KurtpersC1,MeanDC0noPers, SDDC0noPers,SKDC0noPers,KurtDC0noPers,MeanBC1noPers,SDBC1noPers,SKBC1noPers,KurtBC1noPers,MeanDC1noPers,SDDC1noPers,SKDC1noPers,KurtDC1noPers,SUMPersC1noPers,MeanPersC1noPers,SDPersC1noPers,SKPersC1noPers, KurtPersC1noPers])
    return ECGFeat    

In [ ]:
#Random Forest Here



#Build and train first RF on first window size
#make sure to include all features in one csv, including labels
RFFeatures = pd.read_csv("Features/Features.csv",index_col=0)
labels = np.array(RFFeatures["LABEL"])
RFFeatures = RFFeatures.drop('LABEL', axis = 1)
feature_list = list(RFFeatures.columns)
features = np.array(RFFeatures)
rf = RandomForestClassifier(bootstrap=True,max_depth=10,max_features="sqrt",min_samples_leaf=2,min_samples_split=5,n_estimators=600,random_state=42)
rf.fit(RFFeatures,labels)

#Build and train second RF on different window size
RFFeatures2 = pd.read_csv("Features/FeaturesWIN50.csv",index_col=0)
labels2 = np.array(RFFeatures2["LABEL"])
RFFeatures2 = RFFeatures2.drop('LABEL', axis = 1)
feature_list2 = list(RFFeatures2.columns)
features2 = np.array(RFFeatures2)
rf2 = RandomForestClassifier(bootstrap=True,max_depth=10,max_features="sqrt",min_samples_leaf=2,min_samples_split=5,n_estimators=600,random_state=42)
rf2.fit(RFFeatures2,labels2)



In [ ]:
#Get list of files

extensions = ['.hea']
List = [fn for fn in os.listdir(cwd+ '\\test') if any(fn.endswith(ext) for ext in extensions)]

#List = sorted(glob.glob('*.hea'))

#initialize global variables
WIN = 250
DIM = 50
step=WIN/DIM
SDtresh = 1
maxdimension = 1

In [ ]:
#Check one ECG at a time, one-step RF
Answers = []
for ecg in List:
    ecg2,b = ecg.split('.')
    DataAndRR = ExtractDataAndRRFeatures(ecg2)
    Data = DataAndRR["Data"]
    RRStats = DataAndRR["RRstats"]
    #CleanData = Clean(Data) #This is the real line
    CleanData = Clean(ecg2+".txt") #Proxy 
    if (NoisyCheck(CleanData,DataAndRR)==1):
        #Bypass
        Answers.append('~')
    else:
        features = np.concatenate((RRStats,extract(CleanData)), axis=None)
        Answers.append(list(rf.predict(features.reshape(1, -1)))[0])
Answers

In [ ]:
#Check one ECG at a time, Two-step RF
Answers = []
for ecg in List:
    ecg2,b = ecg.split('.')
    DataAndRR = ExtractDataAndRRFeatures(ecg2)
    Data = DataAndRR["Data"]
    RRStats = DataAndRR["RRstats"]
    #CleanData = Clean(Data) #This is the real line
    CleanData = Clean(ecg2+".txt") #Proxy for now
    if (NoisyCheck(CleanData,DataAndRR)==1):
        #Bypass
        Answers.append('~')
    else:
        features = np.concatenate((RRStats,extract(CleanData)), axis=None)
        Predtemp = list(rf.predict(features.reshape(1, -1)))[0]
        if (Predtemp=='N'): 
            Answers.append('N')
        else:
            Answers.append(list(rf2.predict(features.reshape(1, -1)))[0])    

Answers         